In [89]:
from threading import activeCount

import numpy as np
# from keras.src.layers import RepeatVector, TimeDistributed44
# noinspection PyUnresolvedReferences

from tensorflow.keras.layers import RepeatVector, TimeDistributed

from torchgen.executorch.api.et_cpp import return_type

from get_data import get_data
# noinspection PyUnresolvedReferences
from tensorflow.keras.layers import Input, Dense, LeakyReLU, LSTM
# noinspection PyUnresolvedReferences

from tensorflow.keras.models import Model


In [90]:

ticker = "AMZN"
feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
start_date = '2017-01-01'
end_date = '2023-01-01'
seq_train_length = 10
steps_to_predict = 2
scale = True
test_size = 0.2
save_data = False
split_by_date = False

d_r = get_data(
	ticker,
	feature_columns,
	start_date,
	end_date,
	seq_train_length,
	steps_to_predict,
	scale,
	test_size,
	save_data,
	split_by_date
)

print(d_r[0].head(3))
# print(d_r[1].head(3))


[*********************100%***********************]  1 of 1 completed

I am read_data
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
after reset index
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
Open
High
Low
Close
Volume
y shape is:  (1499, 2)
calling train_test_split
        Date      Open      High       Low     Close  Adj Close    Volume  \
0 2017-01-03  0.000000  0.000000  0.000000  0.000000  37.683498  0.168315   
1 2017-01-04  0.000157  0.000305  0.002204  0.001179  37.859001  0.103879   
2 2017-01-05  0.001216  0.007843  0.004259  0.008993  39.022499  0.315538   

   date  future_1  future_2  
0     0  0.001179  0.008993  
1     1  0.008993  0.014212  
2     2  0.014212  0.014524  


In [91]:

result_df = d_r[1]
# print(result_df)

x_train = result_df['X_train']
y_train = result_df['y_train']
x_test = result_df['X_test']
y_test = result_df['y_test']

print("X_train shape:", result_df['X_train'].shape)
print("Y_train shape:", result_df['y_train'].shape)
print("X_test shape:", result_df['X_test'].shape)
print("Y_test shape:", result_df['y_test'].shape)

print("First 3 elements of X_train:", result_df['X_train'][:3])
print("First 3 elements of Y_train:", result_df['y_train'][:3])
print("First 3 elements of X_test:", result_df['X_test'][:3])
print("First 3 elements of Y_test:", result_df['y_test'][:3])




X_train shape: (1199, 10, 5)
Y_train shape: (1199, 2)
X_test shape: (300, 10, 5)
Y_test shape: (300, 2)
First 3 elements of X_train: [[[0.07313266 0.07334988 0.07018439 0.06825648 0.30622876]
  [0.06767401 0.07020821 0.06906539 0.07197407 0.1649292 ]
  [0.07383929 0.07522096 0.07747816 0.07711555 0.14611349]
  [0.07735225 0.07729107 0.07944824 0.07692749 0.10635246]
  [0.07492432 0.07553943 0.07647105 0.07539612 0.14351206]
  [0.07364839 0.07267973 0.07209682 0.06948224 0.16776016]
  [0.06814285 0.06856272 0.07017419 0.068777   0.15324828]
  [0.06686024 0.06715944 0.06705798 0.06703744 0.1455779 ]
  [0.06617372 0.0693921  0.07046244 0.07160802 0.11914918]
  [0.06746639 0.06742481 0.07002161 0.06861918 0.11393995]]

 [[0.28032738 0.27804613 0.2830738  0.2816633  0.13941863]
  [0.28200182 0.28173518 0.28629172 0.28690553 0.17572384]
  [0.2848417  0.28425646 0.29017088 0.28726822 0.12345301]
  [0.28568223 0.2827636  0.28828895 0.2851962  0.08815522]
  [0.28334135 0.28279012 0.2890214  0.2

In [92]:
num_time_steps = x_train.shape[1]
num_features = x_train.shape[2]
print(num_time_steps, num_features)
auto_encode_input = Input(shape=(num_time_steps, num_features))


10 5


In [93]:
encoder = LSTM(64, return_sequences = True)(auto_encode_input)
encoder = LSTM(32, return_sequences = True)(encoder)
encoder = LSTM(16, return_sequences = False)(encoder)
# encoder = LSTM()(encoder)

In [94]:
latent_space = Dense(16, activation = "relu")(encoder)



In [95]:
decoder = RepeatVector(num_time_steps)(latent_space)
decoder = LSTM(32, return_sequences = True)(decoder)
decoder = LSTM(64, return_sequences = True)(decoder)
decoder = TimeDistributed(Dense(num_features, activation = "linear"))(decoder)

In [96]:
# decoder = Dense(num_features, activation = "sigmoid")(decoder)

In [97]:
autoecoder = Model(inputs = auto_encode_input, outputs = decoder)
autoecoder.compile(optimizer = "adam", loss = "mse")

In [98]:
autoecoder.fit(x_train, x_train, epochs = 50, batch_size = 32, validation_data = (x_test, x_test))

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 0.1788 - val_loss: 0.0457
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0364 - val_loss: 0.0161
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0137 - val_loss: 0.0074
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0056 - val_loss: 0.0050
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0037 - val_loss: 0.0045
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0035 - val_loss: 0.0044
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0035 - val_loss: 0.0045
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0034 - val_loss: 0.0042
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0032 - val_loss: 0.0037
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0026 - va

In [99]:
encoder_model = Model(inputs = auto_encode_input, outputs = latent_space)
x_test_pred = autoecoder.predict(x_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step


In [100]:
encoded_data = encoder_model.predict(x_test)
print(encoded_data.shape)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
(300, 16)


In [101]:
print(encoded_data)

[[0.00000000e+00 1.22716725e+00 9.66423810e-01 ... 7.06731975e-01
  1.23777294e+00 1.79395843e+00]
 [1.49447657e-03 9.91165638e-02 4.79969867e-02 ... 1.70213774e-01
  1.16614476e-01 1.59232646e-01]
 [0.00000000e+00 1.54019818e-01 1.30985245e-01 ... 1.62578791e-01
  2.17108771e-01 3.13007176e-01]
 ...
 [0.00000000e+00 1.84805661e-01 8.56346041e-02 ... 2.23597601e-01
  2.13350594e-01 3.00899178e-01]
 [0.00000000e+00 1.13277245e+00 1.31807745e-01 ... 1.08564579e+00
  9.38405573e-01 1.47055233e+00]
 [0.00000000e+00 1.04866564e+00 5.80721259e-01 ... 6.73539460e-01
  1.02272427e+00 1.51395082e+00]]


In [102]:
mse = np.mean(np.power(x_test - x_test_pred, 2), axis=(1, 2))
mae = np.mean(np.abs(x_test - x_test_pred), axis=(1, 2))

# Print average reconstruction error
print("Average MSE reconstruction error:", np.mean(mse))
print("Average MAE reconstruction error:", np.mean(mae))

Average MSE reconstruction error: 0.002200648
Average MAE reconstruction error: 0.025314836


In [105]:
# compressed_x_train = encoder_model.predict(x_train)
# compressed_x_test = encoder_model.predict(x_test)

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [106]:
import pickle

compressed_x_train = encoder_model.predict(x_train).reshape(x_train.shape[0], -1)
compressed_x_test = encoder_model.predict(x_test).reshape(x_train.shape[0], -1)

# with open("compressed_x_train.pkl", "wb") as f:
#     pickle.dump(compressed_x_train, f)
# with open("compressed_x_test.pkl", "wb") as f:
#     pickle.dump(compressed_x_test, f)


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
